# Double-Ended Queues
[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/matyama/pfds/blob/main/notebooks/deques.ipynb)

### Overview
|                instance               | persistence | amortization | empty | isEmpty |     cons     | head |     tail     |     snoc     | last |     init     |
|:-------------------------------------:|:-----------:|:------------:|:-----:|:-------:|:------------:|:----:|:------------:|:------------:|:----:|:------------:|
|    Output-Restricted Banker's Deque   |  persistent |      yes     |  O(1) |   O(1)  | O(n) / O(1)* | O(1) | O(n) / O(1)* | O(n) / O(1)* |   -  |       -      |
|   Output-Restricted Real-Time Deque   |  persistent |      no      |  O(1) |   O(1)  |     O(1)     | O(1) |     O(1)     |     O(1)     |   -  |       -      |
| Output-Restricted Hood-Melville Deque |  persistent |      no      |  O(1) |   O(1)  |    O(1)**    | O(1) |     O(1)     |     O(1)     |   -  |       -      |
|             Banker's Deque            |  persistent |      yes     |  O(1) |   O(1)  | O(n) / O(1)* | O(1) | O(n) / O(1)* | O(n) / O(1)* | O(1) | O(n) / O(1)* |
|            Real-Time Deque            |  persistent |      no      |  O(1) |   O(1)  |     O(1)     | O(1) |     O(1)     |     O(1)     | O(1) |     O(1)     |

*** via the `ConsQueue` wrapper*

### Qutput-Restricted Deques
An intermediate step between FIFO queues and *double-ended queue (or deques)* are *output-restricted queues* which, additionaly to ordinary queues, provide a `cons :: a -> q a -> q a` function. Class `OutputRestrictedDeque` thus supports insertion to both ends but inspection and removal only from the front.

Some `Queue` instances can easily be extended with custom `cons` implementations. For instance for the Banker's queue we have
```haskell
cons :: a -> BankersQueue a -> BankersQueue a
cons x (BQ lenf f lenr r) = BQ (lenf + 1) (x:f) lenr r
```

Similarly, the real-time queue prepends new element `x` to `f` but also to `s` to maintain the invariant $|s| = |f| - |r|$:
```haskell
cons :: a -> RealTimeQueue a -> RealTimeQueue a
cons x (RTQ f r s) = RTQ (x:f) r (x:s)
```

On the other hand, for some queues (e.g. the `HoodMelvilleQueue`) such trivial implementation is not possible. Fortunately, it's possible to implement generic wrapper (see `ConstQueue` below) which maintains explicit front list and provides `cons` in $O(1)$ worst case time and other operations with constant overhead over respective operations of an underlying queue.

In [ ]:
class Queue q where
    
    -- | Construct new (empty) queue
    empty :: q a
    
    -- | Check whether a queue is empty
    isEmpty :: q a -> Bool
    
    -- | Append new item to the back of a queue
    snoc :: q a -> a -> q a
    
    -- | Retrieve the head item of a non-empty queue
    head :: q a -> a
    
    -- | Remove the head of a non-empty queue and retrieve the rear
    tail :: q a -> q a

In [ ]:
{-# LANGUAGE FlexibleInstances, FlexibleContexts #-}

class OutputRestrictedDeque q where
    
    -- | Construct new (empty) deque
    empty :: q a
    
    -- | Check whether a deque is empty
    isEmpty :: q a -> Bool
    
    -- Insert, inspect and remove the front element
    cons :: a -> q a -> q a
    head :: q a -> a
    tail :: q a -> q a
    
    -- | Append new item to the back of a queue
    snoc :: q a -> a -> q a


data ConsQueue q a = ConsQueue [a] (q a)

instance Queue q => OutputRestrictedDeque (ConsQueue q) where

    -- | Constructs new queue in O(empty) time for 'empty' of the underlying queue
    empty = ConsQueue [] Ghci2.empty

    -- | Checks whether the queue is empty in O(isEmpty q) time.
    isEmpty (ConsQueue f q) = null f && Ghci2.isEmpty q
    
    -- Prepend new element to the front of the queue in O(1) worst case time.
    cons x (ConsQueue f q) = ConsQueue (x:f) q
    
    -- | Appends new element to the back of the queue in O(snoc q) time.
    snoc (ConsQueue f q) x = ConsQueue f (Ghci2.snoc q x)
    
    -- | Retrieves the head of the queue in O(head q) time.
    head (ConsQueue [] q) = Ghci2.head q
    head (ConsQueue (x:_) _) = x
    
    -- | Removes the head of the queue and returns the rest in O(tail q) time.
    tail (ConsQueue [] q) = ConsQueue [] (Ghci2.tail q)
    tail (ConsQueue (_:f) q) = ConsQueue f q

### Deques
The `Deque` class provides on top of standard `Queue` operations and `cons` two additional operations:
 - `last` peeks the last element of the deque
 - `init` removes the last element of the deque (i.e. returns a queue with all items but the last one)

In [ ]:
class Deque q where

    -- | Construct new (empty) deque
    empty :: q a
    
    -- | Check whether a deque is empty
    isEmpty :: q a -> Bool
    
    -- Insert, inspect and remove the front element
    cons :: a -> q a -> q a
    head :: q a -> a
    tail :: q a -> q a
    
    -- Insert, inspect and remove the rear element
    snoc :: q a -> a -> q a
    last :: q a -> a
    init :: q a -> q a

## Banker's Deque
While standard queues can be berfetly balanced (all elemnts end up in the front part), this is not possible for deques that can be updated from both ends which define perfect balance as having equal distribution of elements between the front and the rear part.

The idea behind Banker's Deque is that a balanced deque maintains follwing invariant (for a constant $c > 0$):
$$
|f| \leq c|r| + 1 \land |r| \leq c|f| + 1
$$
This invariant ensures that any deque is no more off perfect balance than by a factor of $c$. Moreover, it works for singleton deques with the single element being in either the front or the rear part due to the $+1$.

### Representation
`BankersDeque` has the same representation as the `BankersQueue`, that is
 1. length of the front part
 1. the front list
 1. length of the rear part
 1. the rear list

*Note: It might be possible to parametrize the data structure with $c$ but we keep the implementation simple.*

### Balancing
The helper function `check` is responsible for maintaining the balancing invariant. More precisely it checks whether one of the lists is too long and if so it transfers the items onto the back of the other.

*Note: More on the amortized complexity analysis can be found in the book.*

### Reversal
As a bonus there's the `rev` function which takes advantage of the symmetry in the implementation of `check` and trivially reverses a deque in $O(1)$ time. This is quite common property between deques.

Note that one could implement the update operations on the deque via `rev` but the approach show below is slightly faster. Just as an example: `init = rev . tail . rev`.

In [ ]:
-- | BD = (len of front, front, len of rear, rear)
data BankersDeque a = BD Int [a] Int [a]

-- | Balancing coefficient c > 1
balanceCoef = 3 :: Int

-- | Check and possibly re-balance a deque in O(n) worst case and O(1) amortized time.
-- |
-- | This function maintans the invariant: |f| <= c * |r| + 1 and |r| <= c * |f| + 1.
check :: Int -> [a] -> Int -> [a] -> BankersDeque a
check lenf f lenr r
    | lenf > balanceCoef * lenr + 1 =
        let i = (lenf + lenr) `div` 2
            j = lenf + lenr - i
            f' = take i f
            r' = r ++ reverse (drop i f)
        in BD i f' j r'
    | lenr > balanceCoef * lenf + 1 =
        let j = (lenf + lenr) `div` 2
            i = lenf + lenr - j
            r' = take j r
            f' = f ++ reverse (drop j r)
        in BD i f' j r'
    | otherwise = BD lenf f lenr r

-- | Reverse given deque in O(1) worst case time.
rev :: BankersDeque a -> BankersDeque a
rev (BD lenf f lenr r) = BD lenr r lenf f

instance Deque BankersDeque where

    -- | Construct new (empty) deque in O(1) worst case time.
    empty = BD 0 [] 0 []
    
    -- | Check whether a deque is empty in O(1) worst case time.
    isEmpty (BD lenf _ lenr _) = (lenf + lenr) == 0
    
    -- | Prepend new element to the front of the deque in
    -- | O(n) worst case and O(1) amortized time.
    -- |
    -- | Note: Resulting deque is constructed by 'check' which gives the time complexity.
    cons x (BD lenf f lenr r) = check (lenf+1) (x:f) lenr r
    
    -- | Peek the front element of the deque in O(1) worst case time.
    head (BD _ [] _ _) = error "Deque is empty"
    head (BD _ (x:_) _ _) = x
    
    -- | Remove the front element from a non-empty deque in 
    -- | O(n) worst case and O(1) amortized time.
    -- |
    -- | Note: Resulting deque is constructed by 'check' which gives the time complexity.
    tail (BD _ [] _ _) = error "Deque is empty"
    tail (BD lenf (x:f') lenr r) = check (lenf-1) f' lenr r
    
    -- | Append new element to the back of the deque in
    -- | O(n) worst case and O(1) amortized time.
    -- |
    -- | Note: Resulting deque is constructed by 'check' which gives the time complexity.
    snoc (BD lenf f lenr r) x = check lenf f (lenr+1) (x:r)
    
    -- | Peek the rear element of the deque in O(1) worst case time.
    last (BD _ _ _ []) = error "Deque is empty"
    last (BD _ _ _ (x:_)) = x
    
    -- | Remove the rear element from a non-empty deque in
    -- | O(n) worst case and O(1) amortized time.
    -- |
    -- | Note: Resulting deque is constructed by 'check' which gives the time complexity.
    init (BD _ _ _ []) = error "Deque is empty"
    init (BD lenf f lenr (x:r')) = check lenf f (lenr-1) r'

## Real-Time Deque
`RealTimeDeque` is an extension of `RealTimeQueue` which schedules both front and rear list. And as an instance of *lazy rebuilding* and *scheduling* combination, there are two steps to make the data structure:
 1. identify and break monolithic operations into incremental ones
 1. schedule the execution of suspensions from these new operations so that relevant portion is a priori memoized

### Representation
`RealTimeDeque` has similar representation to the `RealTimeQueue` in that it uses schedules and to the `BankersDeque` by tracking list lengths:
 1. length of the front part
 1. the front list
 1. schedule with suspensions of the front list
 1. length of the rear part
 1. the rear list
 1. schedule with suspensions of the rear list

### Balancing
Rotation is implemented by a simple adaptation of `check` for `BankersDeque` that additionally accepts schedules. The main difference is that the monolithic `f ++ reverse (drop j r)` is replaced by an incremental `rotateDrop f j r` (analogously for the symmetric case with `r`).

The trick behind `rotateDrop` is that it performs $1 < c < 4$* `drop` steps per each `++` step and then calls `rotateRev` which in turn runs $c$ steps of `reverse` for every remaining step of `++`.

It can be argued that `rotateDrop` and `rotateRev` still call the expensive `drop` and `reverse` functions. However, this time both `drop` and `reverse` are called with arguments of bounded size and thus run in $O(1)$*.

**See detailed explanation in the book.*

### Scheduling
The constant time complexity of `cons`, `tail`, `snoc` and `last` then follows from the fact that due to correct schedule management all necessary suspensions are memoized prior to calling `check` by calls to `exec1` and `exec2` (depending on the operation).

Note that it's important to manage both the front and the rear schedule at the same time and to ensure that both are completely evaluated before next rotation happens (so that all suspensions that are forced by `rotateDrop` and `rotateRev` are memoized prior to invoking them).

In [ ]:
-- | RTD = (len of front, front, front schedule, len of rear, rear, rear schedule)
data RealTimeDeque a = RTD Int [a] [a] Int [a] [a]

-- | Execute first suspension in given schedule and return remaining schedule in O(1) time.
exec1 :: [a] -> [a]
exec1 (_:s) = s
exec1 s = s

-- | Call 'exec1' twice. Runs in O(1) time.
exec2 :: [a] -> [a]
exec2 = exec1 . exec1

-- | Incementally rotate and reverse at the same time in constant time.
-- |
-- | Note: Each 'drop' and 'reverse' call runs in constant time because the argument has bounded size.
rotateRev :: [a] -> [a] -> [a] -> [a]
rotateRev [] r a = reverse r ++ a
rorateRev (x:f) r a = x : rotateRev f (drop balanceCoef r) (reverse (take balanceCoef r ++ a))

-- | Rotate all but the last 'j' rear elements to the back of the front.
-- |
-- | This is an incremental version of `f ++ reverse (drop j r)` that with proper scheduling
-- | runs in O(1) worst case time.
-- |
-- | Note: Each 'drop' call runs in constant time because the argument has bounded size.
rotateDrop :: [a] -> Int -> [a] -> [a]
rotateDrop f j r =
    if j < balanceCoef then rotateRev f (drop j r) []
    else let (x:f') = f in x : rotateDrop f' (j - balanceCoef) (drop balanceCoef r)

-- | Check and possibly re-balance a deque in O(1) worst case time.
-- |
-- | This is simple adaptation of 'check' for 'BankersDeque' additionally accepts schedules.
-- | The main difference is that `r ++ reverse (drop i f)` is replaced by `rotateDrop r i f`
-- | (analogously for the symmetric case with 'f').
-- |
-- | Note: The constant time complexity follows from the fact that 'RealTimeDeque' uses scheduling
-- |       and all necessary suspensions are memoized prior to calling 'check' by 'exec1' and 'exec2'.
check :: Int -> [a] -> [a] -> Int -> [a] -> [a] -> RealTimeDeque a
check lenf f sf lenr r sr
    | lenf > balanceCoef * lenr + 1 =
        let i = (lenf + lenr) `div` 2
            j = lenf + lenr - i
            f' = take i f
            r' = rotateDrop r i f
        in RTD i f' f' j r' r'
    | lenr > balanceCoef * lenf + 1 =
        let j = (lenf + lenr) `div` 2
            i = lenf + lenr - j
            r' = take j r
            f' = rotateDrop f j r
        in RTD i f' f' j r' r'
    | otherwise = RTD lenf f sf lenr r sr

-- | Reverse given deque in O(1) worst case time.
rev :: RealTimeDeque a -> RealTimeDeque a
rev (RTD lenf f sf lenr r sr) = RTD lenr r sr lenf f sf

instance Deque RealTimeDeque where

    -- | Construct new (empty) deque in O(1) worst case time.
    empty = RTD 0 [] [] 0 [] []
    
    -- | Check whether a deque is empty in O(1) worst case time.
    isEmpty (RTD lenf _ _ lenr _ _) = (lenf + lenr) == 0
    
    -- | Prepend new element to the front of the deque in O(1) worst case time.
    -- |
    -- | Executing one job from both scheudles ensures that all required suspensions are memoized
    -- | prior to calling 'check'.
    cons x (RTD lenf f sf lenr r sr) = check (lenf+1) (x:f) (exec1 sf) lenr r (exec1 sr)
    
    -- | Peek the front element of the deque in O(1) worst case time.
    head (RTD _ [] _ _ [] _) = error "Deque is empty"
    head (RTD _ [] _ _ (x:_) _) = x
    head (RTD _ (x:_) _ _ _ _) = x
    
    -- | Remove the front element from a non-empty deque in O(1) worst case time.
    -- |
    -- | Executing two jobs from both scheudles ensures that all required suspensions are memoized
    -- | prior to calling 'check'.
    tail (RTD _ [] _ _ [] _) = error "Deque is empty"
    tail (RTD _ [] _ _ (_:_) _) = RTD 0 [] [] 0 [] [] -- empty
    tail (RTD lenf (x:f') sf lenr r sr) = check (lenf-1) f' (exec2 sf) lenr r (exec2 sr)
    
    -- | Append new element to the back of the deque in O(1) worst case time.
    -- |
    -- | Executing one job from both scheudles ensures that all required suspensions are memoized
    -- | prior to calling 'check'.
    snoc (RTD lenf f sf lenr r sr) x = check lenf f (exec1 sf) (lenr+1) (x:r) (exec1 sr)
    
    -- | Peek the rear element of the deque in O(1) worst case time.
    last (RTD _ [] _ _ [] _) = error "Deque is empty"
    last (RTD _ (x:_) _ _ [] _) = x
    last (RTD _ _ _ _ (x:_) _) = x
    
    -- | Remove the rear element from a non-empty deque in O(1) worst case time.
    -- |
    -- | Resulting deque is constructed by 'check' which gives the time complexity.
    -- | Executing two jobs from both scheudles ensures that all required suspensions are memoized
    -- | prior to calling 'check'.
    init (RTD _ [] _ _ [] _) = error "Deque is empty"
    init (RTD _ (_:_) _ _ [] _) = RTD 0 [] [] 0 [] [] -- empty
    init (RTD lenf f sf lenr (x:r') sr) = check lenf f (exec2 sf) (lenr-1) r' (exec2 sr)